# 상품 정보 스크래핑

- SSG(온라인 이마트몰)에서 종류 별 상품 정보를 수집한다.
- 상품 정보 : 종류, 브랜드, 상품명, 가격, 용량, 용량 단위, 상품 이미지 url, 원재료표 이미지 url, 스크래핑 출처

In [3]:
# 파이썬 버전 확인
!python --version

# Selenium 다운로드
!pip install selenium
!apt-get update

# chromedriver 다운로드
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [55.4 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [929 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:10 https://ppa.laun

In [4]:
# import modules
import selenium

from selenium import webdriver
from selenium.webdriver.common.by import By

import time

In [17]:
# chromedriver 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

In [6]:
# Dataframe 생성용 배열 선언
product_type = []         # 종류
product_brand = []        # 브랜드
product_name = []         # 상품명
product_price = []        # 가격
product_amount = []       # 용량
product_amount_unit = []  # 용량 단위
product_img = []          # 상품 이미지 URL
product_element_img = []  # 원재료표 이미지 URL
product_url = []          # 스크래핑 출처

In [18]:
origin_url = "https://emart.ssg.com/disp/category.ssg?dispCtgId=6000213319&shpp=ssgem"

In [19]:
import re
from selenium.common.exceptions import NoSuchElementException


driver.get(origin_url)
typ = driver.find_element(By.XPATH, '//*[@id="area_disp_ctg_title"]/h2/a').text
print("#" + typ)

page = 1
repeat = True


while(repeat):
  url = origin_url + "&page=" + str(page)
  print("\n -- url : " + url)
  driver.get(url)
  print(f' -- Change to Page {page} --')

  for i in range(1, 81):
    try:
      driver.find_element(By.XPATH, '//*[@id="ty_thmb_view"]/ul/li['+ str(i) +']/div/a').click()
    except NoSuchElementException as e:
      repeat = False
      print(" -- End Scrapping")
      break

    time.sleep(2)

    print("\n[상품 " + str(i) + "]")

    try:
      brand = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/div[2]/h2/span/div/div/a').text
    except NoSuchElementException as e:
      brand = 'None'

    try:
      name = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/div[2]/h2/span/span').text
    except NoSuchElementException as e:
      name = 'None'

    try:
      driver.find_elements(By.CLASS_NAME, 'ssg_percent')
      price = int(re.sub(r'[^0-9]', '', driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/div[4]/div[2]/div/span[1]/em').text))
    except NoSuchElementException as e:
      try:
        price = int(re.sub(r'[^0-9]', '', driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/div[4]/div[2]/div/span/em').text))
      except NoSuchElementException as e:
        price = int(re.sub(r'[^0-9]', '', driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/div[3]/div[2]/div/span/em').text))


    try:
      amount = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/div[4]/div[2]/p').text.split('총 용량 : ')[1]
    except NoSuchElementException as e:
      try:
        amount = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/div[3]/div[2]/p').text.split('총 용량 : ')[1]
      except IndexError as e:
        amount = 'None'
    except IndexError as e:
      amount = 'None'


    try:
      img = driver.find_element(By.XPATH, '//*[@id="mainImg"]').get_attribute('src')
    except NoSuchElementException as e:
      img = 'None'

    try:
      iframe_element = driver.find_element(By.XPATH, '//*[@id="_ifr_html"]')
      driver.switch_to.frame(iframe_element)  # iframe 내부로 전환
      element_img = driver.find_element(By.XPATH, '//img[@alt="품질표시이미지1"]').get_attribute('src')
    except NoSuchElementException as e:
      element_img = 'None'

    print(f'* 브랜드 : {brand}')
    print(f'* 제품명 : {name}')
    print(f'* 가격 : {price}')
    print(f'* 용량 : {amount}')
    print(f'* 상품 이미지 : {img}')
    print(f'* 성분 이미지 : {element_img}')

    product_type.append(typ)
    product_brand.append(brand)
    product_name.append(name)
    product_price.append(price)
    product_amount.append(amount)
    product_img.append(img)
    product_element_img.append(element_img)
    product_url.append(driver.current_url)

    driver.back()

  page += 1


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[상품 28]
* 브랜드 : 오뚜기
* 제품명 : 오뚜기 컵누들 매콤한맛 컵 (37.8GX6)
* 가격 : 7980
* 용량 : 6개
* 상품 이미지 : https://sitem.ssgcdn.com/08/39/78/item/0000008783908_i1_1100.jpg
* 성분 이미지 : https://sitem.ssgcdn.com/08/39/78/qlty/0000008783908_q1.jpg

[상품 29]
* 브랜드 : 농심
* 제품명 : 농심 배홍동쫄쫄면 129g*4개
* 가격 : 4580
* 용량 : 4개
* 상품 이미지 : https://sitem.ssgcdn.com/39/39/17/item/1000531173939_i1_1100.jpg
* 성분 이미지 : https://sitem.ssgcdn.com/39/39/17/qlty/1000531173939_q1.jpg

[상품 30]
* 브랜드 : None
* 제품명 : 농심 푸팟퐁구리큰사발면 104g
* 가격 : 1650
* 용량 : None
* 상품 이미지 : https://sitem.ssgcdn.com/00/54/69/item/1000589695400_i1_1100.jpg
* 성분 이미지 : None

[상품 31]
* 브랜드 : 자연주의
* 제품명 : 자연주의 유기농 통밀소면 400g
* 가격 : 4980
* 용량 : 400g
* 상품 이미지 : https://sitem.ssgcdn.com/60/89/68/item/0000007688960_i1_1100.jpg
* 성분 이미지 : https://sitem.ssgcdn.com/60/89/68/qlty/0000007688960_q1.jpg

[상품 32]
* 브랜드 : None
* 제품명 : 농심 푸팟퐁구리큰사발면 104g
* 가격 : 1650
* 용량 : None
* 상품 이미지 : https://sitem.ssgcdn.com/00/54/69/item/1000589695400_i1_1100

In [21]:
# dataframe 생성

import pandas as pd

info = {
    "종류" : product_type,
    "브랜드" : product_brand,
    "상품명" : product_name,
    "가격" : product_price,
    "용량" : product_amount,
    "상품 이미지 URL" : product_img,
    "원재료표 이미지 URL" : product_element_img,
    "스크래핑 출처" : product_url
}

product = pd.DataFrame(info)
product

,종류,브랜드,상품명,가격,용량,상품 이미지 URL,원재료표 이미지 URL,스크래핑 출처
0,면류/통조림,하림,하림 The미식 장인라면 담백한맛452g,7800,4개,https://sitem.ssgcdn.com/44/31/97/item/1000257...,https://sitem.ssgcdn.com/44/31/97/qlty/1000257...,https://emart.ssg.com/item/itemView.ssg?itemId...
1,면류/통조림,None,제이쿱 완두콩 400g,1480,None,https://sitem.ssgcdn.com/64/12/04/item/1000580...,None,https://emart.ssg.com/item/itemView.ssg?itemId...
2,면류/통조림,None,[청수] 메밀이 좋아 비빔냉면 280g,6580,280g,https://sitem.ssgcdn.com/72/56/67/item/1000010...,https://sitem.ssgcdn.com/72/56/67/qlty/1000010...,https://emart.ssg.com/item/itemView.ssg?itemId...
3,면류/통조림,None,동원 리챔 프로틴 200g*3,14480,600g,https://sitem.ssgcdn.com/88/65/21/item/1000584...,None,https://emart.ssg.com/item/itemView.ssg?itemId...
4,면류/통조림,dole,[Dole] 후룻바틀 황도 666g,8380,666g,https://sitem.ssgcdn.com/60/81/74/item/0000009...,https://sitem.ssgcdn.com/60/81/74/qlty/0000009...,https://emart.ssg.com/item/itemView.ssg?itemId...
...,...,...,...,...,...,...,...,...
826,면류/통조림,None,벨로띠 칠리페퍼 250g,4980,250g,https://sitem.ssgcdn.com/70/95/30/item/1000574...,https://sitem.ssgcdn.com/70/95/30/qlty/1000574...,https://emart.ssg.com/item/itemView.ssg?itemId...
827,면류/통조림,None,사조 자연산골뱅이 300g,8380,300g,https://sitem.ssgcdn.com/84/20/71/item/1000600...,None,https://emart.ssg.com/item/itemView.ssg?itemId...
828,면류/통조림,오뚜기,오뚜기 꽁치 280g,4480,280g,https://sitem.ssgcdn.com/54/42/79/item/1000064...,https://sitem.ssgcdn.com/54/42/79/qlty/1000064...,https://emart.ssg.com/item/itemView.ssg?itemId...
829,면류/통조림,청정원,청정원 잇츠팜 190g*4입,15980,760g,https://sitem.ssgcdn.com/81/34/40/item/1000593...,https://sitem.ssgcdn.com/81/34/40/qlty/1000593...,https://emart.ssg.com/item/itemView.ssg?itemId...


In [22]:
# 'None'을 None으로 대체
# TO-DO : 추후 제거할 예정

product.replace('None', None, inplace=True)
product

,종류,브랜드,상품명,가격,용량,상품 이미지 URL,원재료표 이미지 URL,스크래핑 출처
0,면류/통조림,하림,하림 The미식 장인라면 담백한맛452g,7800,4개,https://sitem.ssgcdn.com/44/31/97/item/1000257...,https://sitem.ssgcdn.com/44/31/97/qlty/1000257...,https://emart.ssg.com/item/itemView.ssg?itemId...
1,면류/통조림,None,제이쿱 완두콩 400g,1480,None,https://sitem.ssgcdn.com/64/12/04/item/1000580...,None,https://emart.ssg.com/item/itemView.ssg?itemId...
2,면류/통조림,None,[청수] 메밀이 좋아 비빔냉면 280g,6580,280g,https://sitem.ssgcdn.com/72/56/67/item/1000010...,https://sitem.ssgcdn.com/72/56/67/qlty/1000010...,https://emart.ssg.com/item/itemView.ssg?itemId...
3,면류/통조림,None,동원 리챔 프로틴 200g*3,14480,600g,https://sitem.ssgcdn.com/88/65/21/item/1000584...,None,https://emart.ssg.com/item/itemView.ssg?itemId...
4,면류/통조림,dole,[Dole] 후룻바틀 황도 666g,8380,666g,https://sitem.ssgcdn.com/60/81/74/item/0000009...,https://sitem.ssgcdn.com/60/81/74/qlty/0000009...,https://emart.ssg.com/item/itemView.ssg?itemId...
...,...,...,...,...,...,...,...,...
826,면류/통조림,None,벨로띠 칠리페퍼 250g,4980,250g,https://sitem.ssgcdn.com/70/95/30/item/1000574...,https://sitem.ssgcdn.com/70/95/30/qlty/1000574...,https://emart.ssg.com/item/itemView.ssg?itemId...
827,면류/통조림,None,사조 자연산골뱅이 300g,8380,300g,https://sitem.ssgcdn.com/84/20/71/item/1000600...,None,https://emart.ssg.com/item/itemView.ssg?itemId...
828,면류/통조림,오뚜기,오뚜기 꽁치 280g,4480,280g,https://sitem.ssgcdn.com/54/42/79/item/1000064...,https://sitem.ssgcdn.com/54/42/79/qlty/1000064...,https://emart.ssg.com/item/itemView.ssg?itemId...
829,면류/통조림,청정원,청정원 잇츠팜 190g*4입,15980,760g,https://sitem.ssgcdn.com/81/34/40/item/1000593...,https://sitem.ssgcdn.com/81/34/40/qlty/1000593...,https://emart.ssg.com/item/itemView.ssg?itemId...


In [23]:
# 필수 정보 중 None을 포함한 레코드 제거

product = product.dropna(subset=['상품명', '가격', '용량', '상품 이미지 URL', '원재료표 이미지 URL'])
product

,종류,브랜드,상품명,가격,용량,상품 이미지 URL,원재료표 이미지 URL,스크래핑 출처
0,면류/통조림,하림,하림 The미식 장인라면 담백한맛452g,7800,4개,https://sitem.ssgcdn.com/44/31/97/item/1000257...,https://sitem.ssgcdn.com/44/31/97/qlty/1000257...,https://emart.ssg.com/item/itemView.ssg?itemId...
2,면류/통조림,None,[청수] 메밀이 좋아 비빔냉면 280g,6580,280g,https://sitem.ssgcdn.com/72/56/67/item/1000010...,https://sitem.ssgcdn.com/72/56/67/qlty/1000010...,https://emart.ssg.com/item/itemView.ssg?itemId...
4,면류/통조림,dole,[Dole] 후룻바틀 황도 666g,8380,666g,https://sitem.ssgcdn.com/60/81/74/item/0000009...,https://sitem.ssgcdn.com/60/81/74/qlty/0000009...,https://emart.ssg.com/item/itemView.ssg?itemId...
5,면류/통조림,None,하림 챔 할라피뇨 200g,4480,200g,https://sitem.ssgcdn.com/82/89/99/item/1000556...,https://sitem.ssgcdn.com/82/89/99/qlty/1000556...,https://emart.ssg.com/item/itemView.ssg?itemId...
11,면류/통조림,동원,동원 리챔 핫치폴레 200g,5080,200g,https://sitem.ssgcdn.com/65/73/61/item/1000022...,https://sitem.ssgcdn.com/65/73/61/qlty/1000022...,https://emart.ssg.com/item/itemView.ssg?itemId...
...,...,...,...,...,...,...,...,...
825,면류/통조림,동원,동원 미니언즈 카레참치 135g*4입,11480,540g,https://sitem.ssgcdn.com/06/37/37/item/1000278...,https://sitem.ssgcdn.com/06/37/37/qlty/1000278...,https://emart.ssg.com/item/itemView.ssg?itemId...
826,면류/통조림,None,벨로띠 칠리페퍼 250g,4980,250g,https://sitem.ssgcdn.com/70/95/30/item/1000574...,https://sitem.ssgcdn.com/70/95/30/qlty/1000574...,https://emart.ssg.com/item/itemView.ssg?itemId...
828,면류/통조림,오뚜기,오뚜기 꽁치 280g,4480,280g,https://sitem.ssgcdn.com/54/42/79/item/1000064...,https://sitem.ssgcdn.com/54/42/79/qlty/1000064...,https://emart.ssg.com/item/itemView.ssg?itemId...
829,면류/통조림,청정원,청정원 잇츠팜 190g*4입,15980,760g,https://sitem.ssgcdn.com/81/34/40/item/1000593...,https://sitem.ssgcdn.com/81/34/40/qlty/1000593...,https://emart.ssg.com/item/itemView.ssg?itemId...


In [24]:
# 용량 정보 가공 코드 (수치, 단위 분리)

def split_amount(amount):
    match = re.match(r"([\d,]+)([a-zA-Z가-힣]+)", amount)
    if match:
        number = match.group(1).replace(',', '')  # 쉼표 제거
        unit = match.group(2)
        return number, unit
    return None, None

In [25]:
# 용량 정보 가공 진행

product[['용량 수치', '용량 단위']] = product['용량'].apply(lambda x: pd.Series(split_amount(x)))
product

<ipython-input-25-b14a4df2b96a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product[['용량 수치', '용량 단위']] = product['용량'].apply(lambda x: pd.Series(split_amount(x)))
<ipython-input-25-b14a4df2b96a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product[['용량 수치', '용량 단위']] = product['용량'].apply(lambda x: pd.Series(split_amount(x)))


,종류,브랜드,상품명,가격,용량,상품 이미지 URL,원재료표 이미지 URL,스크래핑 출처,용량 수치,용량 단위
0,면류/통조림,하림,하림 The미식 장인라면 담백한맛452g,7800,4개,https://sitem.ssgcdn.com/44/31/97/item/1000257...,https://sitem.ssgcdn.com/44/31/97/qlty/1000257...,https://emart.ssg.com/item/itemView.ssg?itemId...,4,개
2,면류/통조림,None,[청수] 메밀이 좋아 비빔냉면 280g,6580,280g,https://sitem.ssgcdn.com/72/56/67/item/1000010...,https://sitem.ssgcdn.com/72/56/67/qlty/1000010...,https://emart.ssg.com/item/itemView.ssg?itemId...,280,g
4,면류/통조림,dole,[Dole] 후룻바틀 황도 666g,8380,666g,https://sitem.ssgcdn.com/60/81/74/item/0000009...,https://sitem.ssgcdn.com/60/81/74/qlty/0000009...,https://emart.ssg.com/item/itemView.ssg?itemId...,666,g
5,면류/통조림,None,하림 챔 할라피뇨 200g,4480,200g,https://sitem.ssgcdn.com/82/89/99/item/1000556...,https://sitem.ssgcdn.com/82/89/99/qlty/1000556...,https://emart.ssg.com/item/itemView.ssg?itemId...,200,g
11,면류/통조림,동원,동원 리챔 핫치폴레 200g,5080,200g,https://sitem.ssgcdn.com/65/73/61/item/1000022...,https://sitem.ssgcdn.com/65/73/61/qlty/1000022...,https://emart.ssg.com/item/itemView.ssg?itemId...,200,g
...,...,...,...,...,...,...,...,...,...,...
825,면류/통조림,동원,동원 미니언즈 카레참치 135g*4입,11480,540g,https://sitem.ssgcdn.com/06/37/37/item/1000278...,https://sitem.ssgcdn.com/06/37/37/qlty/1000278...,https://emart.ssg.com/item/itemView.ssg?itemId...,540,g
826,면류/통조림,None,벨로띠 칠리페퍼 250g,4980,250g,https://sitem.ssgcdn.com/70/95/30/item/1000574...,https://sitem.ssgcdn.com/70/95/30/qlty/1000574...,https://emart.ssg.com/item/itemView.ssg?itemId...,250,g
828,면류/통조림,오뚜기,오뚜기 꽁치 280g,4480,280g,https://sitem.ssgcdn.com/54/42/79/item/1000064...,https://sitem.ssgcdn.com/54/42/79/qlty/1000064...,https://emart.ssg.com/item/itemView.ssg?itemId...,280,g
829,면류/통조림,청정원,청정원 잇츠팜 190g*4입,15980,760g,https://sitem.ssgcdn.com/81/34/40/item/1000593...,https://sitem.ssgcdn.com/81/34/40/qlty/1000593...,https://emart.ssg.com/item/itemView.ssg?itemId...,760,g


In [27]:
def extract_itemId(url):
    match = re.search(r"itemId=(\d+)", url)
    if match:
        return match.group(1)
    return None

In [29]:
product['id'] = product['스크래핑 출처'].apply(extract_itemId)
product

<ipython-input-29-6cc88bb3d374>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product['id'] = product['스크래핑 출처'].apply(extract_itemId)


,종류,브랜드,상품명,가격,용량,상품 이미지 URL,원재료표 이미지 URL,스크래핑 출처,용량 수치,용량 단위,id
0,면류/통조림,하림,하림 The미식 장인라면 담백한맛452g,7800,4개,https://sitem.ssgcdn.com/44/31/97/item/1000257...,https://sitem.ssgcdn.com/44/31/97/qlty/1000257...,https://emart.ssg.com/item/itemView.ssg?itemId...,4,개,1000257973144
2,면류/통조림,None,[청수] 메밀이 좋아 비빔냉면 280g,6580,280g,https://sitem.ssgcdn.com/72/56/67/item/1000010...,https://sitem.ssgcdn.com/72/56/67/qlty/1000010...,https://emart.ssg.com/item/itemView.ssg?itemId...,280,g,1000010675672
4,면류/통조림,dole,[Dole] 후룻바틀 황도 666g,8380,666g,https://sitem.ssgcdn.com/60/81/74/item/0000009...,https://sitem.ssgcdn.com/60/81/74/qlty/0000009...,https://emart.ssg.com/item/itemView.ssg?itemId...,666,g,0000009748160
5,면류/통조림,None,하림 챔 할라피뇨 200g,4480,200g,https://sitem.ssgcdn.com/82/89/99/item/1000556...,https://sitem.ssgcdn.com/82/89/99/qlty/1000556...,https://emart.ssg.com/item/itemView.ssg?itemId...,200,g,1000556998982
11,면류/통조림,동원,동원 리챔 핫치폴레 200g,5080,200g,https://sitem.ssgcdn.com/65/73/61/item/1000022...,https://sitem.ssgcdn.com/65/73/61/qlty/1000022...,https://emart.ssg.com/item/itemView.ssg?itemId...,200,g,1000022617365
...,...,...,...,...,...,...,...,...,...,...,...
825,면류/통조림,동원,동원 미니언즈 카레참치 135g*4입,11480,540g,https://sitem.ssgcdn.com/06/37/37/item/1000278...,https://sitem.ssgcdn.com/06/37/37/qlty/1000278...,https://emart.ssg.com/item/itemView.ssg?itemId...,540,g,1000278373706
826,면류/통조림,None,벨로띠 칠리페퍼 250g,4980,250g,https://sitem.ssgcdn.com/70/95/30/item/1000574...,https://sitem.ssgcdn.com/70/95/30/qlty/1000574...,https://emart.ssg.com/item/itemView.ssg?itemId...,250,g,1000574309570
828,면류/통조림,오뚜기,오뚜기 꽁치 280g,4480,280g,https://sitem.ssgcdn.com/54/42/79/item/1000064...,https://sitem.ssgcdn.com/54/42/79/qlty/1000064...,https://emart.ssg.com/item/itemView.ssg?itemId...,280,g,1000064794254
829,면류/통조림,청정원,청정원 잇츠팜 190g*4입,15980,760g,https://sitem.ssgcdn.com/81/34/40/item/1000593...,https://sitem.ssgcdn.com/81/34/40/qlty/1000593...,https://emart.ssg.com/item/itemView.ssg?itemId...,760,g,1000593403481


In [30]:
# 수집한 데이터 csv 추출

product.to_csv("product_database.csv", encoding = "UTF-8")